In [361]:
import math
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, scale
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
%matplotlib inline 

In [362]:
!pip install catboost

In [363]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [366]:
train_df = pd.read_excel("/content/drive/My Drive/Hackathon/Train_dataset.xlsx",index_col="City")
test_df = pd.read_excel("/content/drive/My Drive/Hackathon/Test_dataset.xlsx")
cities = test_df.iloc[:,0]
test_df = test_df.set_index("City")
X = train_df.iloc[:,:-1]
y = train_df.iloc[:,-1].values

In [367]:
test_df

,State,Type,Population [2011],Popuation [2001],Sex Ratio,Median Age,Avg Temp,SWM,Toilets Avl,Water Purity,H Index,Female Population,# of hospitals,Foreign Visitors
City,,,,,,,,,,,,,,
Tuensang,Nagaland,T.C,36774.0,NaN,931.0,23.0,10.0,MEDIUM,94.0,114.0,0.253390,34237.0,17.0,2769.0
Lakshmeshwar,Karnataka,T.M.C,36754.0,NaN,934.0,25.0,38.0,HIGH,62.0,160.0,0.192555,34328.0,13.0,636502.0
Zira,Punjab,M.Cl.,36732.0,NaN,883.0,29.0,35.0,HIGH,63.0,105.0,0.887882,32434.0,17.0,242367.0
Yawal,Maharashtra,M.Cl,36706.0,NaN,887.0,26.0,31.0,HIGH,60.0,174.0,0.407838,32558.0,11.0,4408916.0
Thana Bhawan,Uttar Pradesh,N.P.,36669.0,NaN,877.0,28.0,39.0,LOW,92.0,153.0,0.324456,32159.0,23.0,3104060.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Nandaprayag,Uttarakhand,N.P,1641.0,NaN,848.0,27.0,7.0,MEDIUM,12.0,181.0,0.316926,1392.0,4.0,105882.0
Kirtinagar,Uttarakhand,N.P,1517.0,NaN,852.0,28.0,12.0,HIGH,16.0,198.0,0.336852,1292.0,6.0,105882.0
Kedarnath,Uttarakhand,N.P,612.0,NaN,853.0,24.0,9.0,LOW,19.0,189.0,0.723253,522.0,6.0,105882.0


In [368]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 787 entries, Mumbai to Rudauli
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   State              787 non-null    object 
 1   Type               787 non-null    object 
 2   Population [2011]  739 non-null    float64
 3   Popuation [2001]   295 non-null    float64
 4   Sex Ratio          777 non-null    float64
 5   Median Age         769 non-null    float64
 6   Avg Temp           770 non-null    float64
 7   SWM                783 non-null    object 
 8   Toilets Avl        761 non-null    float64
 9   Water Purity       629 non-null    float64
 10  H Index            647 non-null    float64
 11  Female Population  646 non-null    float64
 12  # of hospitals     772 non-null    float64
 13  Foreign Visitors   697 non-null    float64
dtypes: float64(11), object(3)
memory usage: 92.2+ KB


In [369]:
X.head()
test_df

,State,Type,Population [2011],Popuation [2001],Sex Ratio,Median Age,Avg Temp,SWM,Toilets Avl,Water Purity,H Index,Female Population,# of hospitals,Foreign Visitors
City,,,,,,,,,,,,,,
Tuensang,Nagaland,T.C,36774.0,NaN,931.0,23.0,10.0,MEDIUM,94.0,114.0,0.253390,34237.0,17.0,2769.0
Lakshmeshwar,Karnataka,T.M.C,36754.0,NaN,934.0,25.0,38.0,HIGH,62.0,160.0,0.192555,34328.0,13.0,636502.0
Zira,Punjab,M.Cl.,36732.0,NaN,883.0,29.0,35.0,HIGH,63.0,105.0,0.887882,32434.0,17.0,242367.0
Yawal,Maharashtra,M.Cl,36706.0,NaN,887.0,26.0,31.0,HIGH,60.0,174.0,0.407838,32558.0,11.0,4408916.0
Thana Bhawan,Uttar Pradesh,N.P.,36669.0,NaN,877.0,28.0,39.0,LOW,92.0,153.0,0.324456,32159.0,23.0,3104060.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Nandaprayag,Uttarakhand,N.P,1641.0,NaN,848.0,27.0,7.0,MEDIUM,12.0,181.0,0.316926,1392.0,4.0,105882.0
Kirtinagar,Uttarakhand,N.P,1517.0,NaN,852.0,28.0,12.0,HIGH,16.0,198.0,0.336852,1292.0,6.0,105882.0
Kedarnath,Uttarakhand,N.P,612.0,NaN,853.0,24.0,9.0,LOW,19.0,189.0,0.723253,522.0,6.0,105882.0


In [370]:
X.columns

Index(['State', 'Type', 'Population [2011]', 'Popuation [2001]', 'Sex Ratio',
       'Median Age', 'Avg Temp', 'SWM', 'Toilets Avl', 'Water Purity',
       'H Index', 'Female Population', '# of hospitals', 'Foreign Visitors'],
      dtype='object')

In [371]:
dataset = [X,test_df]
for data in dataset:
    data.drop(columns = ['Popuation [2001]'],inplace=True)

In [372]:
X.describe()

,Population [2011],Sex Ratio,Median Age,Avg Temp,Toilets Avl,Water Purity,H Index,Female Population,# of hospitals,Foreign Visitors
count,7.390000e+02,777.000000,769.000000,770.000000,761.000000,629.000000,647.000000,6.460000e+02,772.000000,6.970000e+02
mean,3.102834e+05,905.712999,26.183355,30.941558,96.086728,150.373609,0.497069,2.910011e+05,41.849741,1.457945e+06
std,8.874849e+05,37.018542,2.113063,6.968289,30.532991,29.063767,0.293421,8.354348e+05,29.086939,1.664151e+06
min,3.677600e+04,818.000000,23.000000,5.000000,50.000000,100.000000,0.003074,3.091300e+04,10.000000,7.980000e+02
25%,5.255000e+04,877.000000,24.000000,28.000000,70.000000,125.000000,0.238586,4.514450e+04,18.000000,2.378540e+05
50%,7.910600e+04,906.000000,26.000000,31.000000,92.000000,150.000000,0.507004,8.306750e+04,28.000000,6.365020e+05
75%,2.374765e+05,928.000000,28.000000,36.000000,119.000000,176.000000,0.752517,2.206772e+05,67.000000,3.104060e+06
max,1.244237e+07,1042.000000,32.000000,40.000000,227.000000,200.000000,0.999774,1.092440e+07,159.000000,4.684707e+06


In [373]:
X.head()

,State,Type,Population [2011],Sex Ratio,Median Age,Avg Temp,SWM,Toilets Avl,Water Purity,H Index,Female Population,# of hospitals,Foreign Visitors
City,,,,,,,,,,,,,
Mumbai,Maharashtra,M.C,12442373.0,878.0,23.0,32.0,MEDIUM,219.0,150.0,0.700440,10924403.0,159.0,4408916.0
Delhi,Delhi,M.C,11007835.0,858.0,27.0,30.0,MEDIUM,215.0,196.0,0.920018,9444722.0,148.0,2379169.0
Bangalore,Karnataka,MPUA,8436675.0,936.0,28.0,37.0,HIGH,212.0,102.0,0.097085,7896728.0,123.0,636502.0
Hyderabad,Telangana,MPUA,6809970.0,930.0,23.0,31.0,MEDIUM,217.0,118.0,0.827744,6333272.0,110.0,126078.0
Ahmedabad,Gujarat,MPUA,5570585.0,852.0,29.0,25.0,LOW,227.0,109.0,0.847941,4746138.0,73.0,284973.0


In [374]:
total = X.isnull().sum().sort_values(ascending=False)
percent_1 = X.isnull().sum()/X.isnull().count()*100
percent_2 = (round(percent_1, 1)).sort_values(ascending=False)
missing_data = pd.concat([total, percent_2], axis=1, keys=['Total', '%'])
missing_data

,Total,%
Water Purity,158,20.1
Female Population,141,17.9
H Index,140,17.8
Foreign Visitors,90,11.4
Population [2011],48,6.1
Toilets Avl,26,3.3
Median Age,18,2.3
Avg Temp,17,2.2
# of hospitals,15,1.9
Sex Ratio,10,1.3


In [375]:
dataset = [X,test_df]
for data in dataset:
    data['Population [2011]'] = data['Population [2011]'].fillna((data["Female Population"] * 1000) /data["Sex Ratio"])
    data['Population [2011]'].fillna( method ='ffill', inplace = True)
    data['Population [2011]']=data['Population [2011]'].astype(int)
    data['Female Population'] = data['Female Population'].fillna((data["Population [2011]"] *data["Sex Ratio"]/1000)).astype(int)
    data["Sex Ratio"].fillna( method ='ffill', inplace = True)

In [376]:
X["Sex Ratio"].fillna( method ='ffill', inplace = True)

In [377]:
X["Sex Ratio"].astype(int)
X.info()


<class 'pandas.core.frame.DataFrame'>
Index: 787 entries, Mumbai to Rudauli
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   State              787 non-null    object 
 1   Type               787 non-null    object 
 2   Population [2011]  787 non-null    int64  
 3   Sex Ratio          787 non-null    float64
 4   Median Age         769 non-null    float64
 5   Avg Temp           770 non-null    float64
 6   SWM                783 non-null    object 
 7   Toilets Avl        761 non-null    float64
 8   Water Purity       629 non-null    float64
 9   H Index            647 non-null    float64
 10  Female Population  787 non-null    int64  
 11  # of hospitals     772 non-null    float64
 12  Foreign Visitors   697 non-null    float64
dtypes: float64(8), int64(2), object(3)
memory usage: 86.1+ KB


In [378]:
# X["Median Age"].fillna(X["Median Age"].median(),inplace=True)
# X["Median Age"]=X["Median Age"].astype(int)

In [379]:
dataset = [X,test_df]
for data in dataset:
    data.drop(columns = ["State",'Sex Ratio',"Female Population","Type"],inplace=True)
X

,Population [2011],Median Age,Avg Temp,SWM,Toilets Avl,Water Purity,H Index,# of hospitals,Foreign Visitors
City,,,,,,,,,
Mumbai,12442373,23.0,32.0,MEDIUM,219.0,150.0,0.700440,159.0,4408916.0
Delhi,11007835,27.0,30.0,MEDIUM,215.0,196.0,0.920018,148.0,2379169.0
Bangalore,8436675,28.0,37.0,HIGH,212.0,102.0,0.097085,123.0,636502.0
Hyderabad,6809970,23.0,31.0,MEDIUM,217.0,118.0,0.827744,110.0,126078.0
Ahmedabad,5570585,29.0,25.0,LOW,227.0,109.0,0.847941,73.0,284973.0
...,...,...,...,...,...,...,...,...,...
Lunawada,36954,23.0,28.0,MEDIUM,68.0,103.0,0.035280,19.0,284973.0
Murshidabad,36947,23.0,36.0,MEDIUM,62.0,136.0,0.056394,22.0,1489500.0
Mahe,36828,28.0,28.0,HIGH,98.0,138.0,0.066752,27.0,106153.0


In [380]:
dataset = [X,test_df]
for data in dataset:
    data.drop(columns = ["SWM"],inplace=True)

In [381]:
test_df

,Population [2011],Median Age,Avg Temp,Toilets Avl,Water Purity,H Index,# of hospitals,Foreign Visitors
City,,,,,,,,
Tuensang,36774,23.0,10.0,94.0,114.0,0.253390,17.0,2769.0
Lakshmeshwar,36754,25.0,38.0,62.0,160.0,0.192555,13.0,636502.0
Zira,36732,29.0,35.0,63.0,105.0,0.887882,17.0,242367.0
Yawal,36706,26.0,31.0,60.0,174.0,0.407838,11.0,4408916.0
Thana Bhawan,36669,28.0,39.0,92.0,153.0,0.324456,23.0,3104060.0
...,...,...,...,...,...,...,...,...
Nandaprayag,1641,27.0,7.0,12.0,181.0,0.316926,4.0,105882.0
Kirtinagar,1517,28.0,12.0,16.0,198.0,0.336852,6.0,105882.0
Kedarnath,612,24.0,9.0,19.0,189.0,0.723253,6.0,105882.0


In [382]:
# X["SWM"].describe()
# X["SWM"] = X["SWM"].fillna("HIGH")


In [383]:
# d = {"LOW":0,"MEDIUM":1,"HIGH":2}
# data = [X,test_df]
# for dataset in data:
#     dataset["SWM"] = dataset["SWM"].map(d)

In [384]:
# data= [X,test_df]
# for dataset in data:
#     dataset['Avg Temp'].fillna((dataset['Avg Temp'].median()), inplace=True)
# for dataset in data:
#     dataset['Avg Temp']=dataset['Avg Temp'].astype(int)

In [385]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 787 entries, Mumbai to Rudauli
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Population [2011]  787 non-null    int64  
 1   Median Age         769 non-null    float64
 2   Avg Temp           770 non-null    float64
 3   Toilets Avl        761 non-null    float64
 4   Water Purity       629 non-null    float64
 5   H Index            647 non-null    float64
 6   # of hospitals     772 non-null    float64
 7   Foreign Visitors   697 non-null    float64
dtypes: float64(7), int64(1)
memory usage: 55.3+ KB


In [386]:
# data= [X,test_df]
# for dataset in data:
#     dataset['H Index'].fillna((dataset['H Index'].median()), inplace=True)
#     dataset["# of hospitals"].fillna((dataset['H Index'].mean()), inplace=True)

In [387]:
# X["# of hospitals"] = X["# of hospitals"].astype(int)

In [388]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 787 entries, Mumbai to Rudauli
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Population [2011]  787 non-null    int64  
 1   Median Age         769 non-null    float64
 2   Avg Temp           770 non-null    float64
 3   Toilets Avl        761 non-null    float64
 4   Water Purity       629 non-null    float64
 5   H Index            647 non-null    float64
 6   # of hospitals     772 non-null    float64
 7   Foreign Visitors   697 non-null    float64
dtypes: float64(7), int64(1)
memory usage: 55.3+ KB


In [389]:
data= [X,test_df]
for dataset in data:
    dataset['Water Purity'].fillna((dataset['Water Purity'].mean()), inplace=True)
    dataset['Water Purity'] = dataset['Water Purity'].astype(int)
    dataset['Foreign Visitors'].fillna((dataset['Foreign Visitors'].median()), inplace=True)
    dataset['Foreign Visitors'] = dataset["Foreign Visitors"].astype(int)
    # dataset['Toilets Avl'].fillna((dataset['Toilets Avl'].median()), inplace=True)
    # dataset['Toilets Avl'] = dataset["Toilets Avl"].astype(int)

In [390]:
X.head()


,Population [2011],Median Age,Avg Temp,Toilets Avl,Water Purity,H Index,# of hospitals,Foreign Visitors
City,,,,,,,,
Mumbai,12442373,23.0,32.0,219.0,150,0.700440,159.0,4408916
Delhi,11007835,27.0,30.0,215.0,196,0.920018,148.0,2379169
Bangalore,8436675,28.0,37.0,212.0,102,0.097085,123.0,636502
Hyderabad,6809970,23.0,31.0,217.0,118,0.827744,110.0,126078
Ahmedabad,5570585,29.0,25.0,227.0,109,0.847941,73.0,284973


In [391]:
# X_train,X_test,y_train,y_test =  train_test_split(X,y,test_size=0.2,random_state = 27)

In [392]:
scaler = StandardScaler()
cols_to_scale = ['Population [2011]', 'Water Purity', 'Median Age', 'Avg Temp', 
                 'Toilets Avl', '# of hospitals', 'Foreign Visitors']
X.loc[:, cols_to_scale] = scaler.fit_transform(X.loc[:, cols_to_scale])
test_df.loc[:, cols_to_scale] = scaler.transform(test_df.loc[:, cols_to_scale])
X.head()

,Population [2011],Median Age,Avg Temp,Toilets Avl,Water Purity,H Index,# of hospitals,Foreign Visitors
City,,,,,,,,
Mumbai,14.057165,-1.507493,0.151993,4.028237,-0.011501,0.700440,4.030201,1.919059
Delhi,12.395701,0.386726,-0.135208,3.897145,1.760264,0.920018,3.651779,0.639808
Bangalore,9.417815,0.860281,0.869995,3.798826,-1.860300,0.097085,2.791730,-0.458510
Hyderabad,7.533785,-1.507493,0.008392,3.962691,-1.244034,0.827744,2.344504,-0.780206
Ahmedabad,6.098345,1.333835,-0.853211,4.290421,-1.590684,0.847941,1.071631,-0.680062


In [393]:
# ct = ColumnTransformer(transformers = [("encoder",OneHotEncoder(), [0])],remainder="passthrough").fit(X)
# X_mat_train = ct.transform(X)
# X_mat_test = ct.transform(test_df)

In [394]:
from catboost import CatBoostRegressor
regressor = CatBoostRegressor(iterations=1000, depth=10, learning_rate=0.14302024557976781, loss_function='RMSE',reg_lambda = 1.0, od_type = "Iter",od_wait = 10,bagging_temperature = 1000000)
regressor.fit(X,y)

0:	learn: 13915.6930515	total: 20.9ms	remaining: 20.9s
1:	learn: 13040.0162645	total: 34.2ms	remaining: 17s
2:	learn: 12124.5856794	total: 47.2ms	remaining: 15.7s
3:	learn: 11314.0592945	total: 63.4ms	remaining: 15.8s
4:	learn: 10516.8693019	total: 76ms	remaining: 15.1s
5:	learn: 9800.8419131	total: 87.5ms	remaining: 14.5s
6:	learn: 9138.9200135	total: 100ms	remaining: 14.2s
7:	learn: 8621.0034688	total: 113ms	remaining: 13.9s
8:	learn: 7991.4928455	total: 125ms	remaining: 13.7s
9:	learn: 7343.3847365	total: 137ms	remaining: 13.5s
10:	learn: 6854.0801990	total: 148ms	remaining: 13.3s
11:	learn: 6429.5690596	total: 160ms	remaining: 13.2s
12:	learn: 6017.4103726	total: 173ms	remaining: 13.1s
13:	learn: 5627.0822792	total: 185ms	remaining: 13s
14:	learn: 5283.7731826	total: 198ms	remaining: 13s
15:	learn: 4941.3894929	total: 210ms	remaining: 12.9s
16:	learn: 4663.1810011	total: 226ms	remaining: 13.1s
17:	learn: 4388.8836130	total: 238ms	remaining: 13s
18:	learn: 4113.2120296	total: 251ms	

In [395]:
# y_pred_train = regressor.predict(X).astype(int)
y_pred = regressor.predict(test_df).astype(int)

In [401]:
results = pd.DataFrame({"CITY":cities,
                        "Cases": y_pred})

results.to_csv("results.csv",index=False)

In [403]:
results.head(50)

,CITY,Cases
0,Tuensang,2837
1,Lakshmeshwar,2373
2,Zira,2480
3,Yawal,2449
4,Thana Bhawan,2581
5,Ramdurg,2553
6,Pulgaon,2522
7,Sadasivpet,2522
8,Nargund,2110
9,Neem-Ka-Thana,2127


In [396]:
# print(math.sqrt(mean_squared_error(y_train, y_pred_train)))
# print(math.sqrt(mean_squared_error(y_test, y_pred)))

In [397]:
# test_y = pd.DataFrame(data = y_pred,columns=["Predicted"])
# # real_y = pd.DataFrame(data = y_test,columns=["Real"])


In [398]:
# df = test_y.join(real_y)
# df.head(50)


In [198]:
# grid = {'learning_rate': [0.03, 0.1],
#         'depth': [4, 6, 10],
#         'l2_leaf_reg': [1, 3, 5, 7, 9]}

# grid_search_result = regressor.grid_search(grid, 
#                                        X=X_train, 
#                                        y=y_train, 
#                                        plot=True)

In [84]:
    # from xgboost import XGBRegressor
    # regressor = XGBRegressor(silent=False, eval_set = [(X_mat_test, y_test)],eval_mertric = "rmse",early_stopping_rounds=10,
    #                         scale_pos_weight=1,
    #                         learning_rate=0.05,  
    #                         colsample_bytree = 0.3,
    #                             subsample = 0.8,
    #                          min_child_weight=2,
    #                             n_estimators=1000, 
    #                             reg_alpha = 0.3,
    #                             max_depth=4, 
    #                             gamma=5)
    # regressor.fit(X_mat_train, y_train)

[06:37:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.3, early_stopping_rounds=10,
             eval_mertric='rmse',
             eval_set=[(<158x78 sparse matrix of type '<class 'numpy.float64'>'
	with 1580 stored elements in Compressed Sparse Row format>,
                        array([ 2230, 10652,  3598, 14001, 10774, 11254,  2208,  5368,  3877,
        2285,  1997,  2390,  2526,  7521,  2353,  217...
        2188,  2265, 11114,  2630,  2739,  2163,  3092,  2871,  2805,
        9430, 10383,   358, 10485,  2086]))],
             gamma=5, importance_type='gain', learning_rate=0.05,
             max_delta_step=0, max_depth=4, min_child_weight=2, missing=None,
             n_estimators=1000, n_jobs=1, nthread=None, objective='reg:linear',
             random_state=0, reg_alpha=0.3, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=False, subsample=0.8, verbosity=1)

In [200]:
# from sklearn.ensemble import GradientBoostingRegressor
# regressor = GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
#                           init=None, learning_rate=0.15, loss='ls', max_depth=3,
#                           max_features=None, max_leaf_nodes=None,
#                           min_impurity_decrease=0.0, min_impurity_split=None,
#                           min_samples_leaf=1, min_samples_split=2,
#                           min_weight_fraction_leaf=0.0, n_estimators=1000,
#                           n_iter_no_change=None, presort='deprecated',
#                           random_state=None, subsample=1.0, tol=0.0001,
#                           validation_fraction=0.1, verbose=0, warm_start=False)
# regressor.fit(X_train, y_train)

In [201]:

# from xgboost import XGBRFRegressor
# regressor = XGBRFRegressor()
# regressor.fit(X_train, y_train)